### 🔧 FOSS Project Embedding Pipeline (Multimodel Ingestion to Weaviate)

This notebook outlines the complete pipeline for embedding Free and Open Source Software (FOSS) project metadata (names and descriptions) using nine different transformer models. The resulting vector representations are ingested into a local Weaviate instance for semantic search and similarity-based retrieval.

### 🎯 Goal

To enable robust semantic matching of CVEs (Common Vulnerabilities and Exposures) to relevant FOSS projects by populating a vector database with diverse model embeddings of project descriptions and names.

---

### 🚀 Pipeline Overview

1. **Data Ingestion**
   - Load FOSS project metadata, specifically the `name` and `description` fields, from a structured dataset (e.g., JSON, CSV).
   - Clean and preprocess the text (e.g., lowercasing, trimming, deduplication, handling nulls).

2. **Model Embedding**
   - Apply **nine transformer models** (e.g., from HuggingFace or Sentence Transformers) to generate vector representations of each FOSS project:
     - Each model processes both the `name` and `description` separately.
     - Models may include general-purpose encoders, domain-specific variants, or multilingual models.

3. **Vector Storage in Weaviate**
   - Connect to a **locally running Weaviate instance** with vector index enabled.
   - Define a Weaviate schema with one class (e.g., `FOSSProject`) and multiple vector fields (e.g., `name_vector_model1`, `desc_vector_model1`, etc.).
   - Insert project entries into Weaviate, including:
     - Original metadata (name, description).
     - All generated vectors with model-specific field names.
     - Optional: metadata such as model used, processing time, or source info.

4. **Search and Validation**
   - Use Weaviate's vector search (`nearVector`) or hybrid search to validate embeddings by querying semantically similar entries.
   - Optionally log distance or certainty metadata for inspection.

---

### 🧠 Key Benefits

- **Multimodel Embedding**: Redundancy and diversity in embeddings improves robustness for downstream tasks (e.g., matching CVEs with fuzzy descriptions).
- **Modular Architecture**: Easy to add/remove models, change Weaviate schema, or plug into external CVE parsing pipelines.
- **Product-Ready**: Designed for local reproducibility and scalable production use, with extensibility for distributed Weaviate or inference acceleration.


In [1]:
import sys
from pathlib import Path

project_root = Path().resolve()
while not (project_root / 'embedding_pipeline').exists() and project_root != project_root.parent:
    project_root = project_root.parent

sys.path.append(str(project_root))


In [2]:
from pathlib import Path
from embedding_pipeline.weaviate_db.weaviate_write_operations import create_data_objects

foss_name_description_json: Path = Path("../../json_github_data_cleaned/github_repositories_final_ordered.json")

data_objects = create_data_objects(foss_name_description_json)


processing & embedding freeCodeCamp freeCodeCamp...
embedded freeCodeCamp freeCodeCamp successfully
embedded freeCodeCamp freeCodeCamp description successfully
appended freeCodeCamp freeCodeCamp to data objects successfully
processing & embedding codecrafters-io build-your-own-x...
embedded codecrafters-io build-your-own-x successfully
embedded codecrafters-io build-your-own-x description successfully
appended codecrafters-io build-your-own-x to data objects successfully
processing & embedding sindresorhus awesome...
embedded sindresorhus awesome successfully
embedded sindresorhus awesome description successfully
appended sindresorhus awesome to data objects successfully
processing & embedding EbookFoundation free-programming-books...
embedded EbookFoundation free-programming-books successfully
embedded EbookFoundation free-programming-books description successfully
appended EbookFoundation free-programming-books to data objects successfully
processing & embedding public-apis public-ap

In [3]:
from embedding_pipeline.weaviate_db.weaviate_config import connect_to_local_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import verify_weaviate_client_ready
from embedding_pipeline.weaviate_db.weaviate_config import close_weaviate_client
from embedding_pipeline.weaviate_db.weaviate_config import list_weaviate_collections, inspect_collection_properties, retrieve_existing_weaviate_collection

local_client = connect_to_local_weaviate_client()

list_weaviate_collections(local_client)

close_weaviate_client(local_client)


Connecting to local client...
Connected to local weaviate client--> is ready: True
Collection names:
- FOSS_vectors
Collections: FOSS_vectors
Closing connection to weaviate client


In [ ]:
local_client = connect_to_local_weaviate_client()


foss_collection = retrieve_existing_weaviate_collection("FOSS_vectors")
inspect_collection_properties(local_client,foss_collection)

close_weaviate_client(local_client)